# FFNN

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense

In [21]:
def read_table(path):
    """Reads table and drops irrelevant rows.
    Returns an analysis ready DataFrame"""
    df = pd.read_table('EMG/'+path)
    df = df[df['class']!=0]
    df = df[df['class'] != 7]
    return df

In [22]:
df = read_table('1/2_1.txt')

for n in range(2,37):
    path = f'{n}/2_{n}.txt'
    df = pd.concat([df, read_table(path)])

In [23]:
df.drop(columns=['time'], inplace=True)
df.reset_index(drop=True,inplace=True)

# Map the classes so they range from 0-6 for cross entropy loss to work
mapping = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6}
df['class'] = df['class'].map(mapping)

In [24]:
df['class'].value_counts()

class
3    127509
5    127490
4    125881
2    124058
1    121604
0    121539
Name: count, dtype: int64

In [45]:
X = df.drop(columns=['class'])
y = df['class']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [59]:
train, test = train_test_split(df, test_size=0.20)

In [62]:
train = train.sort_values(by=['class'])
train

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
173312,0.00000,0.00001,0.00000,-0.00001,0.00000,-0.00001,-0.00001,0.00000,0
203064,-0.00002,0.00001,0.00001,0.00002,0.00002,-0.00001,0.00000,0.00000,0
545647,0.00000,0.00000,-0.00002,-0.00001,0.00000,0.00014,0.00000,0.00001,0
395450,-0.00002,0.00000,-0.00003,0.00003,0.00004,-0.00001,-0.00001,-0.00003,0
547036,-0.00001,-0.00003,-0.00001,0.00001,-0.00002,0.00009,0.00000,0.00000,0
...,...,...,...,...,...,...,...,...,...
214827,0.00016,0.00000,-0.00003,0.00008,0.00098,0.00020,-0.00004,0.00016,6
225727,-0.00022,-0.00010,-0.00013,-0.00019,-0.00003,-0.00006,0.00012,-0.00020,6
214160,-0.00020,-0.00003,0.00010,0.00037,-0.00089,-0.00097,-0.00009,0.00008,6
214338,-0.00036,0.00001,0.00004,-0.00002,-0.00031,0.00046,-0.00001,0.00094,6


In [47]:
def scaler(df):
    """Normalizes the data, returns a DataFrame with the new scaled numbers"""
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(scaled, columns=df.columns)
    return df_scaled

In [48]:
X_train = scaler(X_train)
X_test = scaler(X_test)

In [49]:
X_train

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
0,-0.010359,-0.002928,-0.078963,-0.039704,0.060978,0.004830,0.056641,0.004176
1,-0.166956,-0.149361,-0.526426,-3.833352,-2.258048,-0.233341,-0.055375,-0.190832
2,0.616025,1.388181,2.456666,3.491019,3.709579,3.974337,1.400833,0.686706
3,0.094038,0.363153,1.263429,0.035418,-0.155465,-0.789072,-0.279406,0.052929
4,-0.949937,-0.002928,-0.078963,-0.189947,-0.495588,-0.233341,-0.279406,-0.337088
...,...,...,...,...,...,...,...,...
603747,-0.323552,-0.295793,-0.302694,1.462731,-0.031783,0.322390,0.392689,-0.288336
603748,-0.636744,-0.222577,0.443079,0.335905,-1.670562,0.044525,0.504705,0.247937
603749,-0.114757,-0.076144,-0.004385,0.072979,0.030058,0.004830,-0.111383,0.004176
603750,-0.219154,-0.661874,-2.465436,-2.218234,-0.155465,-0.074560,0.112649,-0.093328


In [31]:
model = Sequential()
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                90        
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 7)                 77        
                                                                 
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=50)

Epoch 1/20
12076/12076 [==============================] - 8s 636us/step - loss: 1.1738 - accuracy: 0.5468
Epoch 2/20
12076/12076 [==============================] - 6s 532us/step - loss: 1.0224 - accuracy: 0.6151
Epoch 3/20
12076/12076 [==============================] - 6s 506us/step - loss: 1.0032 - accuracy: 0.6174
Epoch 4/20
12076/12076 [==============================] - 8s 669us/step - loss: 0.9963 - accuracy: 0.6180
Epoch 5/20
12076/12076 [==============================] - 6s 506us/step - loss: 0.9918 - accuracy: 0.6201
Epoch 6/20
12076/12076 [==============================] - 6s 502us/step - loss: 0.9823 - accuracy: 0.6250
Epoch 7/20
12076/12076 [==============================] - 6s 501us/step - loss: 0.9764 - accuracy: 0.6271
Epoch 8/20
12076/12076 [==============================] - 6s 507us/step - loss: 0.9743 - accuracy: 0.6279
Epoch 9/20
12076/12076 [==============================] - 6s 502us/step - loss: 0.9725 - accuracy: 0.6295
Epoch 10/20
12076/12076 [=====================

In [16]:
model.evaluate(X_test,y_test)

4717/4717 [==============================] - 2s 428us/step - loss: 0.9780 - accuracy: 0.6196


[0.9779634475708008, 0.6195987462997437]

In [ ]:
df = read_table('1/2_1.txt')
for n in range(2,37):
    path = f'{n}/2_{n}.txt'
    df = pd.concat([df, read_table(path)])

# LSTM

In [63]:
df_time[]

,time,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
0,880,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
1,881,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
2,882,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
3,883,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
4,884,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
...,...,...,...,...,...,...,...,...,...,...
754685,50226,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
754686,50227,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
754687,50228,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
754688,50229,0.00014,0.00003,-0.00001,0.00001,-0.00012,0.00006,-0.00007,-0.00013,6


In [17]:
df = pd.read_table('EMG/1/1_1.txt')
df = df[df['class']!=0]
df = 


,time,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
2287,2400,-0.00001,0.00000,-0.00001,0.00000,0.00000,-0.00001,-0.00001,0.00001,1
2288,2401,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1
2289,2402,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1
2290,2403,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1
2291,2404,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1
...,...,...,...,...,...,...,...,...,...,...
61574,63912,-0.00012,-0.00011,0.00004,0.00000,0.00001,0.00011,0.00005,-0.00005,6
61575,63913,-0.00012,-0.00011,0.00004,0.00000,0.00001,0.00011,0.00005,-0.00005,6
61576,63914,-0.00012,-0.00011,0.00004,0.00000,0.00001,0.00011,0.00005,-0.00005,6
61577,63915,-0.00012,-0.00011,0.00004,0.00000,0.00001,0.00011,0.00005,-0.00005,6
